In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.types import Row
from datetime import datetime

In [4]:
sc = SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/18 06:18:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark=SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option","some-value").getOrCreate()

In [6]:
srecord=sc.parallelize([
    Row(roll_no=1,name="Chandru",passed=True,marks={'Math':89,'Physics':87,'Chemistry':96},sports=['chess','football'],DoB=datetime(2012,5,1,12,1,5)),
    Row(roll_no=2,name="Arsath",passed=False,marks={'Math':95,'Physics':66,'Chemistry':77},sports=['carrom','tennis'],DoB=datetime(2012,5,12,14,2,5)),
    Row(roll_no=3,name="Bhuboo",passed=True,marks={'Math':95,'Physics':100,'Chemistry':95},sports=['football','kabadi'],DoB=datetime(2012,5,14,12,2,5))
])

In [7]:
srdf=srecord.toDF()

In [8]:
srdf.show()

+-------+-------+------+--------------------+------------------+-------------------+
|roll_no|   name|passed|               marks|            sports|                DoB|
+-------+-------+------+--------------------+------------------+-------------------+
|      1|Chandru|  true|{Math -> 89, Chem...| [chess, football]|2012-05-01 12:01:05|
|      2| Arsath| false|{Math -> 95, Chem...|  [carrom, tennis]|2012-05-12 14:02:05|
|      3| Bhuboo|  true|{Math -> 95, Chem...|[football, kabadi]|2012-05-14 12:02:05|
+-------+-------+------+--------------------+------------------+-------------------+



In [9]:
srdf.createOrReplaceTempView('records')

In [10]:
spark.sql("SELECT * FROM records").show()

+-------+-------+------+--------------------+------------------+-------------------+
|roll_no|   name|passed|               marks|            sports|                DoB|
+-------+-------+------+--------------------+------------------+-------------------+
|      1|Chandru|  true|{Math -> 89, Chem...| [chess, football]|2012-05-01 12:01:05|
|      2| Arsath| false|{Math -> 95, Chem...|  [carrom, tennis]|2012-05-12 14:02:05|
|      3| Bhuboo|  true|{Math -> 95, Chem...|[football, kabadi]|2012-05-14 12:02:05|
+-------+-------+------+--------------------+------------------+-------------------+



In [11]:
re=spark.sql("SELECT * FROM records")
type(re)

pyspark.sql.dataframe.DataFrame

24/11/18 06:18:46 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [12]:
spark.sql('SELECT roll_no,marks["Physics"],sports[1] FROM records').show()

+-------+--------------+---------+
|roll_no|marks[Physics]|sports[1]|
+-------+--------------+---------+
|      1|            87| football|
|      2|            66|   tennis|
|      3|           100|   kabadi|
+-------+--------------+---------+



In [13]:
spark.sql("SELECT * FROM records where passed= True").show()

+-------+-------+------+--------------------+------------------+-------------------+
|roll_no|   name|passed|               marks|            sports|                DoB|
+-------+-------+------+--------------------+------------------+-------------------+
|      1|Chandru|  true|{Math -> 89, Chem...| [chess, football]|2012-05-01 12:01:05|
|      3| Bhuboo|  true|{Math -> 95, Chem...|[football, kabadi]|2012-05-14 12:02:05|
+-------+-------+------+--------------------+------------------+-------------------+



In [14]:
spark.sql('SELECT * FROM records where marks["Chemistry"]<40').show()

+-------+----+------+-----+------+---+
|roll_no|name|passed|marks|sports|DoB|
+-------+----+------+-----+------+---+
+-------+----+------+-----+------+---+



In [15]:
srdf.createGlobalTempView('globalrecord')

In [16]:
spark.sql("SELECT * FROM global_temp.globalrecord").show()

+-------+-------+------+--------------------+------------------+-------------------+
|roll_no|   name|passed|               marks|            sports|                DoB|
+-------+-------+------+--------------------+------------------+-------------------+
|      1|Chandru|  true|{Math -> 89, Chem...| [chess, football]|2012-05-01 12:01:05|
|      2| Arsath| false|{Math -> 95, Chem...|  [carrom, tennis]|2012-05-12 14:02:05|
|      3| Bhuboo|  true|{Math -> 95, Chem...|[football, kabadi]|2012-05-14 12:02:05|
+-------+-------+------+--------------------+------------------+-------------------+



In [17]:
srdf.columns

['roll_no', 'name', 'passed', 'marks', 'sports', 'DoB']

In [18]:
srdf=srdf.drop('passed')

In [19]:
spark.sql("SELECT round((marks.Physics+marks.Chemistry+marks.Math)/3)avg_marks FROM records").show()

+---------+
|avg_marks|
+---------+
|     91.0|
|     79.0|
|     97.0|
+---------+



In [20]:
srdf=spark.sql("SELECT *,round((marks.Physics+marks.Chemistry+marks.Math)/3)avg_marks FROM records")

In [21]:
srdf.show()

+-------+-------+------+--------------------+------------------+-------------------+---------+
|roll_no|   name|passed|               marks|            sports|                DoB|avg_marks|
+-------+-------+------+--------------------+------------------+-------------------+---------+
|      1|Chandru|  true|{Math -> 89, Chem...| [chess, football]|2012-05-01 12:01:05|     91.0|
|      2| Arsath| false|{Math -> 95, Chem...|  [carrom, tennis]|2012-05-12 14:02:05|     79.0|
|      3|Gowtham|  true|{Math -> 95, Chem...|[football, kabadi]|2012-05-14 12:02:05|     97.0|
+-------+-------+------+--------------------+------------------+-------------------+---------+

